# EDA

In [ ]:
from surprise import SVD, Dataset
from surprise.model_selection import cross_validate

# Load the built-in movielens-100k dataset
data = Dataset.load_builtin('ml-100k')

# Use the SVD algorithm
algo = SVD()

# Run 5-fold cross-validation
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)